In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

from tensorflow import keras
### hack tf-keras to appear as top level keras
import sys
sys.modules['keras'] = keras
### end of hack

import keras.backend as K
from keras.utils import multi_gpu_model
from keras.callbacks import TensorBoard

from sklearn.model_selection import KFold

import h5py
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, callbacks
from keras.callbacks import *
from keras.models import Sequential
from keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD
import numpy as np
import pprint
import os
from os import listdir
from os.path import isfile, join
from datetime import datetime

# Importing custom framework
import sys, os
sys.path.insert(0, os.path.abspath('../..'))
from data_split import prepare_data_spectrogram

In [2]:
def build_cnn():
    model = models.Sequential()

    model.add(layers.Conv2D(227, (3, 1), activation='relu', input_shape=(227, 227, 1)))
    model.add(layers.MaxPooling2D((3, 1), 2))
    
    model.add(layers.Conv2D(64, (3, 1), activation='relu'))
    model.add(layers.MaxPooling2D((2, 1), 2))
    
    model.add(layers.Conv2D(128, (3, 1), activation='relu'))
    model.add(layers.MaxPooling2D((2, 1), 2))
    
    model.add(layers.Conv2D(128, (3, 1), activation='relu'))
    model.add(layers.MaxPooling2D((2, 1), 2))
    
    model.add(layers.Conv2D(128, (3, 1), activation='relu'))
    model.add(layers.MaxPooling2D((2, 1), 2))
    
    #Fully Connected Part
    model.add(layers.Flatten())
    
    model.add(layers.Dense(1024))#, input_dim=1024
    model.add(layers.Dropout(0.5, seed=1337))
    model.add(layers.Activation('relu'))
    
    model.add(layers.Dense(512))#, input_dim=1024
    model.add(layers.Dropout(0.5, seed=1337))
    model.add(layers.Activation('relu'))
    
    model.add(layers.Dense(10, input_dim=512))
    model.add(layers.Activation('softmax'))
    return model

In [3]:
spectrogramPath = '../../merged/spectrogram.hdf5'
X_train, X_val, X_test, y_train, y_val, y_test = prepare_data_spectrogram(spectrogramPath, False)
X_train = tf.transpose(X_train, [0, 2, 3, 1])
X_val = tf.transpose(X_val, [0, 2, 3, 1])
X_test = tf.transpose(X_test, [0, 2, 3, 1])

In [4]:
# Decrease training data size to make training feasable
#X_temp = []
#y_temp = []
#n = 300
#count_dict = dict.fromkeys(range(0, 10), 0)
#for i in range(len(y_train)):
#    curr = y_train[i]
#    if count_dict[curr] >= n:
#        finished = True
#        for x in count_dict.values():
#            if x < 300:
#                finished = False
#                break
#        if finished == True:
##            break
#    else:
#        count_dict[curr] = count_dict[curr] + 1
#        X_temp.append(X_train[i])
#        y_temp.append(curr)
        
#X_train = np.array(X_temp)
#y_train = np.array(y_temp)
print(X_train.shape)

(18000, 227, 227, 1)


In [5]:
#tf.reset_default_graph()

tbCallBack = TensorBoard(log_dir='logs', histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         write_images=True)

earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

opt = SGD(lr=0.001, momentum=0.9, clipvalue=5)

# Experiment 1

In [6]:
K.clear_session()
model = build_cnn()
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

filepath = "logs/epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

training_history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val),batch_size=32, callbacks=[earlyStopping, tbCallBack], shuffle=False)
# Generate generalization metrics
#scores = model.evaluate(X_val, y_val, verbose=0)
#print(f'Score {scores}')

    
#val_scores = model.evaluate(X_val, y_val, verbose=0)
#print(val_score)

Epoch 1/100
563/563 [==============================] - 1096s 2s/step - loss: 2.3090 - accuracy: 0.1369 - val_loss: 2.2844 - val_accuracy: 0.2370
Epoch 2/100
563/563 [==============================] - 1098s 2s/step - loss: 2.1454 - accuracy: 0.2280 - val_loss: 1.5899 - val_accuracy: 0.4503
Epoch 3/100
563/563 [==============================] - 1113s 2s/step - loss: 1.6153 - accuracy: 0.4158 - val_loss: 1.4646 - val_accuracy: 0.4663
Epoch 4/100
563/563 [==============================] - 1108s 2s/step - loss: 1.3019 - accuracy: 0.5321 - val_loss: 1.4942 - val_accuracy: 0.4695
Epoch 5/100
563/563 [==============================] - 1112s 2s/step - loss: 1.0107 - accuracy: 0.6398 - val_loss: 1.0531 - val_accuracy: 0.6160
Epoch 6/100
563/563 [==============================] - 1111s 2s/step - loss: 0.7501 - accuracy: 0.7361 - val_loss: 0.9644 - val_accuracy: 0.6498
Epoch 7/100
563/563 [==============================] - 1109s 2s/step - loss: 0.5825 - accuracy: 0.8008 - val_loss: 0.8513 - val_ac

In [8]:
print(model.evaluate(X_test, y_test, verbose=0))

[0.49310582876205444, 0.8511666655540466]


In [9]:
tbCallBack = TensorBoard(log_dir='logs', histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         write_images=True)

earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8)

opt = SGD(lr=0.0001, momentum=0.9, clipvalue=5)
K.clear_session()
model = build_cnn()
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

filepath = "logs/epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

training_history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val),batch_size=32, callbacks=[earlyStopping, tbCallBack], shuffle=False)
print(model.evaluate(X_test, y_test, verbose=0))

Epoch 1/100
563/563 [==============================] - 1112s 2s/step - loss: 2.3378 - accuracy: 0.0999 - val_loss: 2.3006 - val_accuracy: 0.1097
Epoch 2/100
563/563 [==============================] - 1107s 2s/step - loss: 2.3046 - accuracy: 0.1053 - val_loss: 2.2969 - val_accuracy: 0.1493
Epoch 3/100
563/563 [==============================] - 1108s 2s/step - loss: 2.3014 - accuracy: 0.1062 - val_loss: 2.2933 - val_accuracy: 0.1388
Epoch 4/100
563/563 [==============================] - 1107s 2s/step - loss: 2.2990 - accuracy: 0.1113 - val_loss: 2.2939 - val_accuracy: 0.1052
Epoch 5/100
563/563 [==============================] - 1109s 2s/step - loss: 2.2932 - accuracy: 0.1215 - val_loss: 2.2765 - val_accuracy: 0.2422
Epoch 6/100
563/563 [==============================] - 1109s 2s/step - loss: 2.2748 - accuracy: 0.1446 - val_loss: 2.2293 - val_accuracy: 0.3527
Epoch 7/100
563/563 [==============================] - 1109s 2s/step - loss: 2.2259 - accuracy: 0.1786 - val_loss: 2.1115 - val_ac

In [10]:
tbCallBack = TensorBoard(log_dir='logs', histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         write_images=True)

earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8)

opt = keras.optimizers.Adam(learning_rate=0.001)
K.clear_session()
model = build_cnn()
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

filepath = "logs/Ex3Epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

training_history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val),batch_size=32, callbacks=[earlyStopping, tbCallBack], shuffle=False)
print(model.evaluate(X_test, y_test, verbose=0))

Epoch 1/100
563/563 [==============================] - 1115s 2s/step - loss: 1.8048 - accuracy: 0.3218 - val_loss: 1.3985 - val_accuracy: 0.5412
Epoch 2/100
563/563 [==============================] - 1114s 2s/step - loss: 0.5836 - accuracy: 0.7991 - val_loss: 0.4918 - val_accuracy: 0.8280
Epoch 3/100
563/563 [==============================] - 1116s 2s/step - loss: 0.2978 - accuracy: 0.9027 - val_loss: 0.5302 - val_accuracy: 0.8100
Epoch 4/100
563/563 [==============================] - 1115s 2s/step - loss: 0.2046 - accuracy: 0.9341 - val_loss: 0.7322 - val_accuracy: 0.8107
Epoch 5/100
563/563 [==============================] - 1114s 2s/step - loss: 0.1964 - accuracy: 0.9353 - val_loss: 0.8702 - val_accuracy: 0.7768
Epoch 6/100
563/563 [==============================] - 1115s 2s/step - loss: 0.1522 - accuracy: 0.9507 - val_loss: 0.7877 - val_accuracy: 0.7947
Epoch 7/100
563/563 [==============================] - 1114s 2s/step - loss: 0.1319 - accuracy: 0.9576 - val_loss: 0.6663 - val_ac

In [11]:
tbCallBack = TensorBoard(log_dir='logs', histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         write_images=True)

earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

opt = keras.optimizers.Adam(learning_rate=0.0001)
K.clear_session()
model = build_cnn()
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

filepath = "logs/Ex4Epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

training_history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val),batch_size=32, callbacks=[earlyStopping, tbCallBack], shuffle=False)
print(model.evaluate(X_test, y_test, verbose=0))

Epoch 1/100
563/563 [==============================] - 1114s 2s/step - loss: 1.9394 - accuracy: 0.2859 - val_loss: 0.9532 - val_accuracy: 0.6633
Epoch 2/100
563/563 [==============================] - 1117s 2s/step - loss: 0.6759 - accuracy: 0.7660 - val_loss: 0.5231 - val_accuracy: 0.8202
Epoch 3/100
563/563 [==============================] - 1118s 2s/step - loss: 0.3717 - accuracy: 0.8736 - val_loss: 0.6636 - val_accuracy: 0.7730
Epoch 4/100
563/563 [==============================] - 1117s 2s/step - loss: 0.2702 - accuracy: 0.9076 - val_loss: 0.5346 - val_accuracy: 0.8212
Epoch 5/100
563/563 [==============================] - 1118s 2s/step - loss: 0.2295 - accuracy: 0.9223 - val_loss: 0.7860 - val_accuracy: 0.7603
Epoch 6/100
563/563 [==============================] - 1118s 2s/step - loss: 0.1813 - accuracy: 0.9393 - val_loss: 0.6054 - val_accuracy: 0.8073
[0.5084832906723022, 0.8388333320617676]


In [13]:
tbCallBack = TensorBoard(log_dir='logs', histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         write_images=True)

earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

opt = keras.optimizers.Adam(learning_rate=0.01)
K.clear_session()
model = build_cnn()
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

filepath = "logs/Ex5Epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

training_history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val),batch_size=32, callbacks=[earlyStopping, tbCallBack], shuffle=False)
print(model.evaluate(X_test, y_test, verbose=0))

Epoch 1/100
563/563 [==============================] - 1109s 2s/step - loss: 3.5799 - accuracy: 0.0879 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 2/100
563/563 [==============================] - 1110s 2s/step - loss: 2.3037 - accuracy: 0.0838 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 3/100
563/563 [==============================] - 1110s 2s/step - loss: 2.3036 - accuracy: 0.0810 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 4/100
563/563 [==============================] - 1111s 2s/step - loss: 2.3036 - accuracy: 0.0814 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 5/100
563/563 [==============================] - 1111s 2s/step - loss: 2.3036 - accuracy: 0.0814 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 6/100
563/563 [==============================] - 1110s 2s/step - loss: 2.3036 - accuracy: 0.0814 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 7/100
563/563 [==============================] - 1111s 2s/step - loss: 2.3036 - accuracy: 0.0814 - val_loss: 2.3026 - val_ac